# Protein Design Using ResidueSelectors and TaskOperations in PyRosetta

#### Presenter: Jason Klima (klimaj@uw.edu)

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
import os
import py3Dmol
import pyrosetta
import pyrosetta.toolbox
import pyrosetta.distributed.io
import site

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Initialize PyRosetta with custom command line flags:


In [ ]:
pyrosetta.init("-ignore_unrecognized_res 1 -ex1 -ex2aro")

### For this tutorial, let's again use the native protein crambin from PDB ID 1AB1 (http://www.rcsb.org/structure/1AB1)
### Setup the input pose and scorefunction:

In [ ]:
start_pose = pyrosetta.toolbox.rcsb.pose_from_rcsb("1AB1", ATOM=True, CRYS=False)
pose = start_pose.clone()
scorefxn = pyrosetta.create_score_function("ref2015_cart.wts")

### To list the cysteine residues in crambin, now we use ResidueSelectors:
#### Note difference in residue names "CYS:disulfide" vs. "CYS"

In [ ]:
disulfide_res = pyrosetta.rosetta.core.select.residue_selector.ResidueNameSelector("CYS:disulfide")
print(pyrosetta.rosetta.core.select.get_residues_from_subset(disulfide_res.apply(pose)))

### Inspect the starting pose using the `py3Dmol` module. By default, disulfide bonds are not drawn even if they exist in PyRosetta:

In [ ]:
viewer = py3Dmol.view(1200, 800)
viewer.addModels(pyrosetta.distributed.io.to_pdbstring(start_pose), "pdb")
viewer.setStyle({"cartoon": {"color": "spectrum"}, "stick": {"radius": 0.25}})
for i in pyrosetta.rosetta.core.select.get_residues_from_subset(disulfide_res.apply(start_pose)):
    viewer.addResLabels({"chain": "A", "resi": str(i)})
viewer.zoomTo()
viewer.show()

### You can also use this shortcut to view a pose in `py3Dmol` which recognizes disulfide bonds in PyRosetta:

In [ ]:
# Run one of the following, whether you git cloned the pyrosetta_scripts repository or downloaded visualization.py from the Google Drive
#%run ~/pyrosetta_scripts/pilot/apps/klimaj/py3Dmol/visualization.py
#%run /Volumes/coding/Users/coding/Downloads/pyrosetta_scripts/pilot/apps/klimaj/py3Dmol/visualization.py
#%run ./visualization.py

view_pose(pose)

## *Design strategy:* Design away the cysteine residues (i.e. disulfide bonds) using ResidueSelectors and TaskOperations, allowing all side-chains to re-pack and all backbone and side-chain torsions to minimize using the `FastDesign` mover.

### Note: because we did not include the `-detect_disulfide 0` command line flag in `pyrosetta.init()`, in order to mutate disulfide bonded cysteine residues using packing steps in the `FastDesign` mover, first we need to break the disulfide bonds:

In [ ]:
for i in pyrosetta.rosetta.core.select.get_residues_from_subset(disulfide_res.apply(pose)):
    for j in pyrosetta.rosetta.core.select.get_residues_from_subset(disulfide_res.apply(pose)):
        if pyrosetta.rosetta.core.conformation.is_disulfide_bond(pose.conformation(), i, j):
            pyrosetta.rosetta.core.conformation.break_disulfide(pose.conformation(), i, j)

### Now we can declare a ResidueSelector for cysteine residues:

In [ ]:
cys_res = pyrosetta.rosetta.core.select.residue_selector.ResidueNameSelector("CYS")
print(pyrosetta.rosetta.core.select.get_residues_from_subset(cys_res.apply(pose)))

### Note: the pose consists of only chain "A"

In [ ]:
print(pose.pdb_info())

### Although the pose contains only chain A, if the pose contained other chains we could pare the selection to only select chain A and cysteine residues using an `AndResidueSelector`. We will use this ResidueSelector with `RestrictAbsentCanonicalAASRLT` (restrict absent canonical amino acids residue level task operation).

In [ ]:
chain_A = pyrosetta.rosetta.core.select.residue_selector.ChainSelector("A")
print(pyrosetta.rosetta.core.select.get_residues_from_subset(chain_A.apply(pose)))
chain_A_cys_res = pyrosetta.rosetta.core.select.residue_selector.AndResidueSelector(selector1=cys_res, selector2=chain_A)
print(pyrosetta.rosetta.core.select.get_residues_from_subset(chain_A_cys_res.apply(pose)))

### To quickly view the selection from a ResidueSelector, let's show only sticks corresponding to the `chain_A_cys_res` ResidueSelector using the `py3Dmol` module:

In [ ]:
view_pose(pose, chain_A_cys_res)

### Specify a `NotResidueSelector` selecting everything except the `chain_A_cys_res` ResidueSelector to use with a `RestrictToRepackingRLT` (restrict to repacking residue level task operation):

In [ ]:
not_chain_A_cys_res = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector(chain_A_cys_res)
print(pyrosetta.rosetta.core.select.get_residues_from_subset(not_chain_A_cys_res.apply(pose)))

### Quickly view the `not_chain_A_cys_res` ResidueSelector:

In [ ]:
view_pose(pose, not_chain_A_cys_res)

### Now we can set up the TaskOperations for `FastDesign`:

In [ ]:
# The task factory accepts all the task operations
tf = pyrosetta.rosetta.core.pack.task.TaskFactory()

# These are pretty standard
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())


# Disable design (i.e. repack only) on not_chain_A_cys_res
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
    pyrosetta.rosetta.core.pack.task.operation.RestrictToRepackingRLT(), not_chain_A_cys_res))

# Enable design on chain_A_cys_res
aa_to_design = pyrosetta.rosetta.core.pack.task.operation.RestrictAbsentCanonicalAASRLT()
aa_to_design.aas_to_keep("ADEFGHIKLMNPQRSTVWY")
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
    aa_to_design, chain_A_cys_res))

# Convert the task factory into a PackerTask
packer_task = tf.create_task_and_apply_taskoperations(pose)
# View the PackerTask
print(packer_task)

### This time we will set up a `MoveMap` to establish which torsions are free to minimize

In [ ]:
mm = pyrosetta.rosetta.core.kinematics.MoveMap()
mm.set_bb(True)
mm.set_chi(True)
mm.set_jump(True)

### This time we will also setup a different `relaxscript` that optimizes the `ramp_repack_min` weights to prevent too many alanines from being designed (demonstrated at Pre-RosettaCON 2018): 

In [ ]:
relaxscript = pyrosetta.rosetta.std.vector_std_string()
with open(os.path.join(site.getsitepackages()[0], "pyrosetta/database/sampling/relax_scripts/rosettacon2018.txt")) as f:
    lines = f.readlines()
    for line in lines:
        relaxscript.append(" ".join(line.split()))
print(relaxscript)

### Set up `FastDesign`

In [ ]:
fast_design = pyrosetta.rosetta.protocols.denovo_design.movers.FastDesign(scorefxn_in=scorefxn, standard_repeats=1)
fast_design.cartesian(True)
fast_design.set_task_factory(tf)
fast_design.set_movemap(mm)
fast_design.set_script_from_lines(relaxscript)
fast_design.minimize_bond_angles(True)
fast_design.minimize_bond_lengths(True)

### Run the `FastDesign` mover. Note: this takes ~39.6 s

In [ ]:
%time fast_design.apply(pose)

### Inspect the resulting design!

In [ ]:
view_pose(pose)

## *Design Challenge*:

### Copy and modify this script to:
 - Repack all residues and allow all degrees of freedom to minimize (i.e. FastRelax) `start_pose` prior to downstream design. Use the resulting `start_pose` for analysis.
 - Re-design the cysteine residues in `start_pose` to allow only hydrophobic residues excluding glycine and proline
 - Only re-pack residues within a 6 Angstroms radius around the neighborhood of each cysteine residue (Cα-Cα distance)
   - Note this will require using:
   - The NeighborhoodResidueSelector: `pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector(selector, distance, include_focus_in_subset)`
   - The prevent repacking residue level task operation: `pyrosetta.rosetta.core.pack.task.operation.PreventRepackingRLT()`
 - Minimize only the side-chains (chi degrees of freedom) being re-packed or designed
 - Allow the backbone torsions to minimize, except for the first, last and any proline phi/psi angles
 
## *Extra Challenge!*

### Instead of using `pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset` TaskOperations, use `pyrosetta.rosetta.protocols.task_operations.ResfileCommandOperation` TaskOperations by applying resfile-style commands to ResidueSelectors. Example:
 > repack_only_task_operation = pyrosetta.rosetta.protocols.task_operations.ResfileCommandOperation()   
 > repack_only_task_operation.set_command("POLAR EMPTY NC R2 NC T6 NC OP5")   
 > repack_only_task_operation.set_residue_selector(selector)   
 > tf.push_back(repack_only_task_operation)

### By how many Angstroms RMSD did the backbone Cα atoms move?

### What is the delta `total_score` from `start_pose` to `pose`?

### What is the per-residue energy difference for each mutated position between `start_pose` and `pose`?